In [25]:
import pandas as pd
import numpy as np

In [26]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

In [27]:
import re
from collections import Counter

In [28]:
from itertools import zip_longest

In [29]:
df = pd.read_csv("email_marketing_products.csv")
df

,name,rating,reviews,entry_level_price,users,industries,market_segment,pros_themes,pros_counts,cons_themes,cons_counts
0,Constant Contact,4.1,7225,Entry Level Price: Starting at $12.00,Marketing Manager; Owner,Non-Profit Organization Management; Marketing ...,70% Small-Business; 23% Mid-Market,Ease of Use; Easy Creation; Email Marketing; S...,"1,060; 494; 485; 457; 420; 1,060; 494; 485; 45...",Missing Features; Limited Customization; Limit...,280; 255; 246; 212; 185; 280; 255; 246; 212; 185
1,Brevo Marketing Platform,4.5,2477,Entry Level Price: Free,CEO; Founder,Marketing and Advertising; Information Technol...,84% Small-Business; 12% Mid-Market,Ease of Use; Features; Email Marketing; Custom...,439; 266; 248; 203; 198; 439; 266; 248; 203; 198,Missing Features; Limited Features; Limited Cu...,131; 108; 78; 76; 73; 131; 108; 78; 76; 73
2,Intuit Mailchimp Email Marketing,4.3,12859,Entry Level Price: Free,Owner; Marketing Manager,Marketing and Advertising; Non-Profit Organiza...,72% Small-Business; 22% Mid-Market,Ease of Use; Features; Email Marketing; Easy C...,133; 69; 64; 53; 47; 133; 69; 64; 53; 47,Expensive; Learning Curve; Limited Features; L...,38; 34; 30; 23; 22; 38; 34; 30; 23; 22
3,Instantly,4.8,3941,Free trial available,Founder; CEO,Marketing and Advertising; Computer Software,94% Small-Business; 4% Mid-Market,Ease of Use; Customer Support; Helpful; Featur...,"1,826; 1,331; 1,300; 999; 810; 1,826; 1,331; 1...",Missing Features; Expensive; Email Management;...,421; 411; 252; 218; 215; 421; 411; 252; 218; 215
4,Systeme.io,4.8,738,Entry Level Price: Free,Owner; CEO,Marketing and Advertising; Professional Traini...,97% Small-Business; 1% Mid-Market,Ease of Use; Customer Support; Affordable; Fre...,171; 167; 76; 64; 55; 171; 167; 76; 64; 55,Learning Curve; Template Limitations; Limited ...,40; 31; 26; 21; 17; 40; 31; 26; 21; 17
...,...,...,...,...,...,...,...,...,...,...,...
220,Hello Email,4.8,3,NaN,67% Small-Business; 33% Enterprise,67% Small-Business; 33% Enterprise,67% Small-Business; 33% Enterprise,Affordable; Customer Support; Customizability;...,1; 1; 1; 1; 1; 1; 1; 1; 1; 1,NaN,NaN
221,Liana®Cloud Email Marketing,3.2,3,Entry Level Price: Starting at $90.00,100% Small-Business,100% Small-Business,100% Small-Business,NaN,NaN,NaN,NaN
222,LuxSci,4.7,83,Entry Level Price: Contact Us,Mental Health Care; Hospital & Health Care,Mental Health Care; Hospital & Health Care,77% Small-Business; 19% Mid-Market,Customer Support; Security; Ease of Use; HIPAA...,21; 12; 11; 10; 9; 21; 12; 11; 10; 9,Email Issues; Not User-Friendly; UX Improvemen...,5; 5; 5; 4; 4; 5; 5; 5; 4; 4
223,Mailing Manager,4.2,3,Entry Level Price: £45 + VAT per month,33% Enterprise; 33% Mid-Market,33% Enterprise; 33% Mid-Market,33% Enterprise; 33% Mid-Market,NaN,NaN,NaN,NaN


In [30]:
print("Shape:", df.shape)
display(df.dtypes)

Shape: (225, 11)


name                  object
rating               float64
reviews                int64
entry_level_price     object
users                 object
industries            object
market_segment        object
pros_themes           object
pros_counts           object
cons_themes           object
cons_counts           object
dtype: object

In [31]:
print("Unique companies (name):", df["name"].nunique())
print("Duplicate name rows:", df.duplicated(subset=["name"]).sum())

Unique companies (name): 225
Duplicate name rows: 0


In [32]:
key_cols = [
    "name","rating","reviews","entry_level_price","users","industries","market_segment",
    "pros_themes","pros_counts","cons_themes","cons_counts"
]
missing = df[key_cols].isna().sum().sort_values(ascending=False)
display(missing)

entry_level_price    124
cons_themes           80
cons_counts           80
pros_themes           68
pros_counts           68
name                   0
rating                 0
reviews                0
users                  0
industries             0
market_segment         0
dtype: int64

In [33]:
df[key_cols].isna().sum(axis=1).sort_values(ascending=False).head(10)

224    5
182    5
179    5
178    5
175    5
165    5
164    5
157    5
155    5
154    5
dtype: int64

In [34]:
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df["reviews"] = pd.to_numeric(df["reviews"], errors="coerce")

In [35]:
display(df[["rating", "reviews"]].describe())

,rating,reviews
count,225.000000,225.000000
mean,4.366222,243.537778
std,0.437542,1059.062068
min,1.800000,3.000000
25%,4.200000,10.000000
50%,4.400000,21.000000
75%,4.700000,75.000000
max,5.000000,12859.000000


In [36]:
def count_items(s):
    if pd.isna(s) or str(s).strip() == "":
        return 0
    return len([x for x in str(s).split(";") if x.strip()])

df["pros_themes_n"] = df["pros_themes"].apply(count_items)
df["pros_counts_n"] = df["pros_counts"].apply(count_items)

df["cons_themes_n"] = df["cons_themes"].apply(count_items)
df["cons_counts_n"] = df["cons_counts"].apply(count_items)

pros_mismatch = df[df["pros_themes_n"] != df["pros_counts_n"]][
    ["name", "pros_themes_n", "pros_counts_n"]
]

cons_mismatch = df[df["cons_themes_n"] != df["cons_counts_n"]][
    ["name", "cons_themes_n", "cons_counts_n"]
]

print("Pros mismatch rows:", len(pros_mismatch))
print("Cons mismatch rows:", len(cons_mismatch))

pros_mismatch.head(), cons_mismatch.head()

Pros mismatch rows: 0
Cons mismatch rows: 0


(Empty DataFrame
 Columns: [name, pros_themes_n, pros_counts_n]
 Index: [],
 Empty DataFrame
 Columns: [name, cons_themes_n, cons_counts_n]
 Index: [])

In [38]:
df["has_pros_data"] = df["pros_themes"].notna() & df["pros_counts"].notna()
df["has_cons_data"] = df["cons_themes"].notna() & df["cons_counts"].notna()

df["has_any_feedback"] = df["has_pros_data"] | df["has_cons_data"]
df["has_price_data"] = df["entry_level_price"].notna()

summary = {
    "total_products": len(df),
    "with_pros_data": df["has_pros_data"].sum(),
    "with_cons_data": df["has_cons_data"].sum(),
    "with_any_feedback": df["has_any_feedback"].sum(),
    "with_price_data": df["has_price_data"].sum(),
}

summary

{'total_products': 225,
 'with_pros_data': np.int64(157),
 'with_cons_data': np.int64(145),
 'with_any_feedback': np.int64(160),
 'with_price_data': np.int64(101)}

In [39]:
df

,name,rating,reviews,entry_level_price,users,industries,market_segment,pros_themes,pros_counts,cons_themes,cons_counts,pros_themes_n,pros_counts_n,cons_themes_n,cons_counts_n,has_pros_data,has_cons_data,has_any_feedback,has_price_data
0,Constant Contact,4.1,7225,Entry Level Price: Starting at $12.00,Marketing Manager; Owner,Non-Profit Organization Management; Marketing ...,70% Small-Business; 23% Mid-Market,Ease of Use; Easy Creation; Email Marketing; S...,"1,060; 494; 485; 457; 420; 1,060; 494; 485; 45...",Missing Features; Limited Customization; Limit...,280; 255; 246; 212; 185; 280; 255; 246; 212; 185,10,10,10,10,True,True,True,True
1,Brevo Marketing Platform,4.5,2477,Entry Level Price: Free,CEO; Founder,Marketing and Advertising; Information Technol...,84% Small-Business; 12% Mid-Market,Ease of Use; Features; Email Marketing; Custom...,439; 266; 248; 203; 198; 439; 266; 248; 203; 198,Missing Features; Limited Features; Limited Cu...,131; 108; 78; 76; 73; 131; 108; 78; 76; 73,10,10,10,10,True,True,True,True
2,Intuit Mailchimp Email Marketing,4.3,12859,Entry Level Price: Free,Owner; Marketing Manager,Marketing and Advertising; Non-Profit Organiza...,72% Small-Business; 22% Mid-Market,Ease of Use; Features; Email Marketing; Easy C...,133; 69; 64; 53; 47; 133; 69; 64; 53; 47,Expensive; Learning Curve; Limited Features; L...,38; 34; 30; 23; 22; 38; 34; 30; 23; 22,10,10,10,10,True,True,True,True
3,Instantly,4.8,3941,Free trial available,Founder; CEO,Marketing and Advertising; Computer Software,94% Small-Business; 4% Mid-Market,Ease of Use; Customer Support; Helpful; Featur...,"1,826; 1,331; 1,300; 999; 810; 1,826; 1,331; 1...",Missing Features; Expensive; Email Management;...,421; 411; 252; 218; 215; 421; 411; 252; 218; 215,10,10,10,10,True,True,True,True
4,Systeme.io,4.8,738,Entry Level Price: Free,Owner; CEO,Marketing and Advertising; Professional Traini...,97% Small-Business; 1% Mid-Market,Ease of Use; Customer Support; Affordable; Fre...,171; 167; 76; 64; 55; 171; 167; 76; 64; 55,Learning Curve; Template Limitations; Limited ...,40; 31; 26; 21; 17; 40; 31; 26; 21; 17,10,10,10,10,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Hello Email,4.8,3,NaN,67% Small-Business; 33% Enterprise,67% Small-Business; 33% Enterprise,67% Small-Business; 33% Enterprise,Affordable; Customer Support; Customizability;...,1; 1; 1; 1; 1; 1; 1; 1; 1; 1,NaN,NaN,10,10,0,0,True,False,True,False
221,Liana®Cloud Email Marketing,3.2,3,Entry Level Price: Starting at $90.00,100% Small-Business,100% Small-Business,100% Small-Business,NaN,NaN,NaN,NaN,0,0,0,0,False,False,False,True
222,LuxSci,4.7,83,Entry Level Price: Contact Us,Mental Health Care; Hospital & Health Care,Mental Health Care; Hospital & Health Care,77% Small-Business; 19% Mid-Market,Customer Support; Security; Ease of Use; HIPAA...,21; 12; 11; 10; 9; 21; 12; 11; 10; 9,Email Issues; Not User-Friendly; UX Improvemen...,5; 5; 5; 4; 4; 5; 5; 5; 4; 4,10,10,10,10,True,True,True,True
223,Mailing Manager,4.2,3,Entry Level Price: £45 + VAT per month,33% Enterprise; 33% Mid-Market,33% Enterprise; 33% Mid-Market,33% Enterprise; 33% Mid-Market,NaN,NaN,NaN,NaN,0,0,0,0,False,False,False,True


In [40]:
MIN_REVIEWS = 50 

In [41]:
df_50 = df[
    (df["reviews"] >= MIN_REVIEWS) &
    (df["has_any_feedback"])
].copy()

In [42]:
print("Total products in original df:", len(df))
print("Products with reviews >= 50 and feedback:", len(df_50))

Total products in original df: 225
Products with reviews >= 50 and feedback: 62


In [43]:
df_50["reviews"].describe()

count       62.000000
mean       838.016129
std       1903.160146
min         53.000000
25%        106.750000
50%        233.000000
75%        760.500000
max      12859.000000
Name: reviews, dtype: float64

In [44]:
def split_themes(s):
    if pd.isna(s) or str(s).strip() == "":
        return []
    return [x.strip() for x in str(s).split(";") if x.strip()]

In [45]:
def split_counts(s):
    if pd.isna(s) or str(s).strip() == "":
        return []
    parts = [p.strip() for p in str(s).split(";") if p.strip()]
    out = []
    for p in parts:
        digits = re.sub(r"[^\d]", "", p)
        out.append(int(digits) if digits else np.nan)
    return out

pro_rows = []

In [46]:
for _, row in df_50.iterrows():
    name = row["name"]
    themes = split_themes(row["pros_themes"])
    counts = split_counts(row["pros_counts"])

    n = min(len(themes), len(counts))

    for i in range(n):
        pro_rows.append({
            "name": name,
            "theme": themes[i],
            "mentions": counts[i],
            "type": "pro"
        })

In [47]:
pros_long = pd.DataFrame(pro_rows)

pros_long.head()

,name,theme,mentions,type
0,Constant Contact,Ease of Use,1060,pro
1,Constant Contact,Easy Creation,494,pro
2,Constant Contact,Email Marketing,485,pro
3,Constant Contact,Simple,457,pro
4,Constant Contact,Helpful,420,pro


In [48]:
con_rows = []

In [49]:

for _, row in df_50.iterrows():
    name = row["name"]
    themes = split_themes(row["cons_themes"])
    counts = split_counts(row["cons_counts"])

    n = min(len(themes), len(counts))

    for i in range(n):
        con_rows.append({
            "name": name,
            "theme": themes[i],
            "mentions": counts[i],
            "type": "con"
        })

In [50]:

cons_long = pd.DataFrame(con_rows)

cons_long.head()

,name,theme,mentions,type
0,Constant Contact,Missing Features,280,con
1,Constant Contact,Limited Customization,255,con
2,Constant Contact,Limited Features,246,con
3,Constant Contact,Limited Templates,212,con
4,Constant Contact,Layout Issues,185,con


In [51]:
cons_long.shape

(576, 4)

In [52]:
pros_long.shape

(614, 4)

In [59]:
cons_summary = (
    cons_long
    .groupby("theme", as_index=False)
    .agg(
        total_mentions=("mentions", "sum"),
        products_affected=("name", "nunique")
    )
    .sort_values("total_mentions", ascending=False)
)

cons_summary.head(15)

,theme,total_mentions,products_affected
51,Missing Features,3458,37
46,Limited Features,2190,24
28,Expensive,1760,24
42,Learning Curve,1304,24
45,Limited Customization,1062,14
25,Email Management,942,16
49,Limited Templates,588,10
41,Lead Management,436,3
40,Layout Issues,370,1
57,Poor Customer Support,314,14


In [60]:
def cooccurrence_with_target(presence_df: pd.DataFrame, target_theme: str, top_n: int = 20) -> pd.DataFrame:
    if target_theme not in presence_df.columns:
        raise ValueError(f"Target theme '{target_theme}' not found in cons_long themes.")

    N = presence_df.shape[0]  # number of products

    target_vec = presence_df[target_theme].astype(int)
    n_target = int(target_vec.sum())

    # how many products have each theme
    n_theme = presence_df.sum(axis=0).astype(int)

    # how many products have both (target AND theme)
    co = presence_df.mul(target_vec, axis=0).sum(axis=0).astype(int)

    # probabilities per theme
    p_target = n_target / N if N else 0
    p_theme = (n_theme / N).astype(float)
    p_both = (co / N).astype(float)

    # lift per theme
    denom = (p_target * p_theme).replace(0, np.nan)
    lift = p_both / denom

    # P(theme | target) per theme
    p_theme_given_target = (co / n_target).astype(float) if n_target else pd.Series(np.nan, index=presence_df.columns)

    out = pd.DataFrame({
        "theme": n_theme.index,
        "products_with_theme": n_theme.values,
        "products_with_target": co.values,
        "p_theme_given_target": p_theme_given_target.values,
        "lift": lift.values
    })

    # remove the target itself
    out = out[out["theme"] != target_theme].copy()

    # sort by co-occurrence strength
    out = out.sort_values(["products_with_target", "lift"], ascending=[False, False])

    return out.head(top_n)

In [61]:
missing_co = cooccurrence_with_target(presence, "Missing Features", top_n=25)
limited_co = cooccurrence_with_target(presence, "Limited Features", top_n=25)

missing_co.head(10), limited_co.head(10)

(                    theme  products_with_theme  products_with_target  \
 46       Limited Features                   24                    17   
 42         Learning Curve                   24                    16   
 25       Email Management                   16                    11   
 28              Expensive                   24                    11   
 24           Email Issues                   11                     9   
 49      Limited Templates                   10                     8   
 45  Limited Customization                   14                     8   
 57  Poor Customer Support                   14                     6   
 71   Template Limitations                    8                     5   
 36     Integration Issues                    4                     4   
 
     p_theme_given_target      lift  
 46              0.459459  1.148649  
 42              0.432432  1.081081  
 25              0.297297  1.114865  
 28              0.297297  0.743243  
 24  

In [64]:
# --- Presence of PRO themes across ALL df_50 ---
pro_presence_all = (
    pros_long
    .assign(present=1)
    .pivot_table(
        index="name",
        columns="theme",
        values="present",
        aggfunc="max",
        fill_value=0
    )
)

# --- Presence of Missing Features across ALL df_50 ---
missing_flag = (
    cons_long
    .assign(is_missing=lambda x: (x["theme"] == "Missing Features").astype(int))
    .groupby("name", as_index=True)["is_missing"]
    .max()
)

# Align both
analysis_df = pro_presence_all.join(missing_flag, how="left").fillna(0)

# --- Proper co-occurrence analysis ---
def pro_cooccurrence_with_missing_correct(df: pd.DataFrame, top_n: int = 20) -> pd.DataFrame:
    N = df.shape[0]
    n_missing = int(df["is_missing"].sum())

    pro_cols = df.columns.drop("is_missing")

    # overall frequency
    p_pro = df[pro_cols].mean(axis=0)

    # frequency given Missing Features
    p_pro_given_missing = df.loc[df["is_missing"] == 1, pro_cols].mean(axis=0)

    lift = (p_pro_given_missing / p_pro).replace([np.inf, -np.inf], np.nan)

    products_with_missing_and_pro = df.loc[df["is_missing"] == 1, pro_cols].sum(axis=0)
    products_with_pro = df[pro_cols].sum(axis=0)

    out = pd.DataFrame({
        "pro_theme": pro_cols,
        "products_with_pro": products_with_pro.values,
        "products_with_missing_and_pro": products_with_missing_and_pro.values,
        "p_pro_given_missing": p_pro_given_missing.values,
        "lift": lift.values
    })

    out = out.sort_values(
        ["products_with_missing_and_pro", "lift"],
        ascending=[False, False]
    )

    return out.head(top_n)

missing_pro_co_fixed = pro_cooccurrence_with_missing_correct(analysis_df, top_n=20)
missing_pro_co_fixed

,pro_theme,products_with_pro,products_with_missing_and_pro,p_pro_given_missing,lift
19,Ease of Use,58,37,1.000000,1.068966
14,Customer Support,36,24,0.648649,1.117117
33,Helpful,28,21,0.567568,1.256757
30,Features,16,13,0.351351,1.361486
27,Email Marketing,20,11,0.297297,0.921622
35,Intuitive,8,7,0.189189,1.466216
20,Easy Creation,9,7,0.189189,1.303303
4,Automation,14,7,0.189189,0.837838
5,Automation AI,16,7,0.189189,0.733108
48,Simple,10,6,0.162162,1.005405
